<a href="https://colab.research.google.com/github/meli1717/Drone-suiveur-de-personne-/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nouvelle section

In [23]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [25]:
import cv2

# Charger une image
img = cv2.imread('/drive/MyDrive/dataset/train/class_a/image1.jpg')

# Obtenir la taille de l'image
hauteur, largeur = img.shape[:2]

print('La taille de l\'image est {}x{}'.format(largeur, hauteur))


La taille de l'image est 255x255


In [26]:
import os
print(os.listdir('/drive/MyDrive/dataset/train'))


['class_a']


In [27]:
# Chargement des données d'entraînement
data = tf.keras.preprocessing.image.ImageDataGenerator()
#num_classes = 1
data = ImageDataGenerator(rescale=1./255)
batch_size = 32

train_data = data.flow_from_directory('/drive/MyDrive/dataset/train', target_size=(255, 255), batch_size=batch_size, class_mode='binary')
val_data = data.flow_from_directory('/drive/MyDrive/dataset/validation', target_size=(255, 255), batch_size=batch_size, class_mode='binary')


Found 452 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [28]:
# Définition du nombre de classes
num_classes = len(train_data.class_indices)
print(num_classes)

1


In [29]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [30]:
# Construction du modèle CNN
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


In [31]:
# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraînement du modèle
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
15/15 [==============================] - 82s 5s/step - loss: 0.0501 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
15/15 [==============================] - 78s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
15/15 [==============================] - 77s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
15/15 [==============================] - 77s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
15/15 [==============================] - 77s 5s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
14/15 [===========================>..] - ETA: 4s - loss: 0.0000e+00 - accuracy: 0.0000e+00

In [ ]:
# Sauvegarde du modèle
model.save('face_recognition_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('face_recognition_model.h5')


In [ ]:
import sys
print(sys.modules.keys())


In [ ]:
import cv2

# Charger une image
#image = cv2.imread('path/to/image.jpg')

# Charger une vidéo
cap = cv2.VideoCapture('/drive/MyDrive/dataset/test/class_c/video.mp4')


In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

def load_features(filename):
    """Charge les caractéristiques faciales à partir d'un fichier .npy"""
    return np.load(filename)

class FaceDatabase:
    def __init__(self, data):
        self.data = data
        
    def compare(self, features, name):
        """Compare les caractéristiques faciales avec celles de l'individu donné"""
        index = self.data[self.data['name'] == name].index[0]
        known_features = np.array(self.data.iloc[index]['features'])
        return np.dot(features, known_features)


melinda_images = []
melinda_dir = '/drive/MyDrive/dataset/melinda'

for filename in os.listdir(melinda_dir):
    if filename.endswith('.jpg'):
        img = cv2.imread(os.path.join(melinda_dir, filename))
        melinda_images.append(img)

# Convertir les images en tableaux NumPy
melinda_features = np.array([cv2.resize(img, (255, 255)) for img in melinda_images])

# Enregistrer les caractéristiques faciales dans un fichier .npy
np.save('melinda_features.npy', melinda_features)

# Charger les caractéristiques faciales de Melinda et d'autres individus
melinda_features = load_features('melinda_features.npy')

# Convertir les caractéristiques de Melinda en un tableau 2D
melinda_features = melinda_features.reshape((1, -1))

# Créer un DataFrame contenant les caractéristiques faciales et les noms de chaque individu
data = {'Name': ['Melinda'],
        'features': [melinda_features]}
df = pd.DataFrame(data)

# Définir la variable db en tant que DataFrame
db = df


In [ ]:
import numpy as np
import pandas as pd
import cv2


# Charger le classifieur de visages
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Pour une image
#image = cv2.imread('/content/drive/MyDrive/Melinda.jpg')
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# Pour une vidéo
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    #  code de reconnaissance faciale ici
    # Pour chaque visage détecté
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w] # extraire le visage de la vidéo
        face = cv2.resize(face, (255, 255)) # redimensionner le visage pour l'adapter au modèle
        face = np.expand_dims(face, axis=0) # ajouter une dimension pour le batch
        face = face / 255.0 # normaliser l'image
    
        features = model.predict(face) # extraire les caractéristiques faciales avec le modèle

        # Comparer les caractéristiques avec celles de
        # Créer un DataFrame avec les caractéristiques et les noms des visages détectés
        features_df = pd.DataFrame({'features': [f.flatten() for f in features], 'Name': ['Melinda'] * len(features)})
        # Concaténer db et features_df en un seul DataFrame
        concat_df = pd.concat([db, features_df], ignore_index=True)
        predictions = concat_df.compare(concat_df[concat_df['Name']=='Melinda'], align_axis=0)
    
        # Afficher le nom de la personne identifiée
        max_index = None
        if len(predictions) > 0:
            max_index = np.argmax(predictions)
        if max_index == 0:
            name = 'Melinda'
        else:
            name = 'Inconnu'
        cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Afficher la vidéo avec les noms des personnes identifiées
    from google.colab.patches import cv2_imshow
    #cv2_imshow('Video', frame)
    cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2_imshow(frame)

    # Sortir si la touche q est pressée
    if cv2.waitKey(1) == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()
